In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/statoilc-core-iceberg-classifier/sample_submission.csv/data/processed/sample_submission.csv
/kaggle/input/statoilc-core-iceberg-classifier/train.json/data/processed/train.json
/kaggle/input/statoilc-core-iceberg-classifier/test.json/data/processed/test.json
statoilc-core-iceberg-classifier



This kernel is specifically is for Beginners who wants t oexperiment building CNN using Keras. By using this kernel, you can expect to get good score and also learn keras. Keras is simple frameworks where we can initilaize the model and keep stacking the layers we want. It makes building deep neural networks very easy.

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab

import plotly.io as pio
pio.renderers.default = 'kaggle' #렌더러 설정

plt.rcParams['figure.figsize'] = 10,10
%matplotlib inline

In [3]:
# Load the data

train = pd.read_json('../input/statoilc-core-iceberg-classifier/train.json/data/processed/train.json')

In [4]:
test = pd.read_json('/kaggle/input/statoilc-core-iceberg-classifier/test.json/data/processed/test.json')

# Intro About the Data

Sentinet -1 sat(satellite) is at about 680km above earth. Sending pulses(파동) of signals at a particular angle of incidence(투사) and then recoding it back. The data we have been given is backscatter coefficient which is the conventional form of backscatter coefficient given by:

$σo (dB) = βo (dB) + 10log10 [ sin(ip) / sin (ic)] $

where
1. ip = is angle of incidence for a particular pixel
2. ic = is angle of incidence for center of the image
3. K = constant

we have been given $σo$ directly in the data. ### Now coming to the features of  $σo$ basically $σo$ varies with the surface on which the signal is scattered from. For example, for a particular angle of incidence, it varies like:

- settlement - 주거지, barren - 불모지


*             WATER...........           SETTLEMENTS........           AGRICULTURE...........          BARREN........

1.**HH:**     -27.001   ................                     2.70252       .................                -12.7952        ................    -17.25790909

2.**HV: **      -28.035      ................            -20.2665             ..................          -21.4471       .................     -20.019

As you can see, the HH componet varies a lot but HV doesn't. I don't have the data for scatter from ship, but being a metal object, it should vary differently as compared to ice object.

### WTF is HH HV?

Ok, so, this is Sentinal Settalite is equivalent to RISTSAT(an indian remote sensing Sat) and they only transmit pings in H polarization, and not n V polarization. Those H-pings gets scattered, objects change their polarization and returns as a mix of H and V. Since Sentinel has only H-transmitter, return signals are of the form of HH and HV only. Don't ask why VV is not given (because Sentinel don't have V-ping transmitter).

- HH, HV는 레이더 신호의 종류를 나타내는 약자. 레이더는 전파 pulses를 보내는 방식을 이용하는데, 이 전파가 진행될 때, 전기장이 특정 방향으로 진동을 한다. 이 진동 방향을 편파 polarization이라고 한다. 수직으로 움직이거나, 수평으로 흔들린다.

- Horizontal Polarization : 전기장이 지표면과 수평으로 진동하는 전파
 
- Vertical Polarization 전기장이 지표면과 수직으로 진동하는 전파

- Ping : 위성이 짧게 전파를 쏘는 것.

- H-ping : 위성이 쏜 수평 편파 신호

- Scattered : 위성이 쏜 H-ping이 땅이나, 바다 같은 물체에 부딪히면 사방으로 흩어짐. 그런데, 부딪혀서 산란될 때, 일부는 H 상태를 유지하지만, 다른 일부는 V 상태로 바뀌어서 돌아올 수 있음. 물체의 재질이나 모양, 거칠기에 따라 편파가 바뀌는 정도가 달라짐. 우리는 이를 통해, 배와 얼음을 구별해낼 것임!

- Sentinel은 H-송신기만 갖고 있기 때문에, 반환신호는 HH와 HV뿐. 

Now coming to features, for the purpose of this demo code, I am extracting all two bands and taking avg of them as 3rd channel to create a 3-channel RGB equivalent.
  

In [5]:
# Generate the training data
# Create 3 bands having HH, HV and avg of both

X_band_1 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train["band_1"]])
X_band_2 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train['band_2']])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis], ((X_band_1 + X_band_2) / 2)[:, :, :, np.newaxis]], axis = -1)

Q1. pd.read_json으로 읽어온 DataFrame의 형태
- 여태껏 tabular 형식의 pd.read_csv로 데이터를 읽어왔었다. 이번에는 json인데, json 형식으로도 tabular 데이터를 표기할 수 있다.
- 위 데이터 파일에서의 JSON은 이러한 형태였다.
```[
  {
    "id": "dfd5f913",
    "band_1": [-27.878361, -27.15416, ...], // 매우 긴 숫자 리스트
    "band_2": [...] // band_1과 유사한 다른 긴 숫자 리스트 (있다고 가정)
    // "is_iceberg": 0 or 1 (타겟 변수, 있을 수도 있음)
  },
  {
    "id": "another_id",
    "band_1": [...],
    "band_2": [...]
    // "is_iceberg": ...
  }
  // ... 많은 객체들
] 
```

- 위 구조를 파악해보자면 다음과 같다. 각 딕셔너리는 하나의 샘플을 의미한다. 딕셔너리 안에 id 키는 샘플의 고유 id를 의미하고, band_1과 band_2 2개의 키가 각 샘플의 고유 피쳐를 의미한다. 즉, 각 샘플별로, id, band1, band2의 컬럼이 존재하는 것이다. 여기에서 band_1은 HH, band_2는 HV에 대한 정보이다. 따라서, json 파일은 각 샘플별로 4개의 컬럼이 존재하는 정보 파일인 것이다.

- 특정 컬럼(또는 피쳐)에 접근하는 것은 csv파일을 읽을 때와 동일하다. train['band1']이라고 하면, band_1 열 전체를 선택하게 된다. 이 열의 각 셀에는 매우 긴 숫자 리스트가 들어있게 된다. (각 샘플별, band_1에 대한 정보)

Q2. 데이터가 75 x 75 픽셀 배열임을 파악하는 방법.
- 주로 데이터 설명서에 나와있지만,
```
length_of_band_1_sample = len(train['band_1'].iloc[0])
print(f"첫 번째 샘플의 band_1 데이터 길이: {length_of_band_1_sample}")
```
를 통해 확인할 수도 있다. band_1의 첫번째 샘플의 길이를 통해서, 총 픽셀의 개수를 알 수가 있다.

Description
```
X_band_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train['band_1']])
```
1. 일단 np.array를 통해, 어떠한 리스트를 numpy array 형식으로 변환하는 것은 알 수 있겠다. 어레이 형식으로 변환하는 이유는, 연산하기에 더욱 편하기 때문이다.

2. 리스트 컴프리헨션을 사용했다.
```for band in train['band_1']```을 한 번 보면, train['band_1']은, 각 샘플에 해당하는 band_1 정보이다. 즉, 각 샘플의 band_1 리스트(band)에 대해서, band를 numpy array형식으로 변환한 후에, 배열의 데이터 타입을 32비트 부동소수점으로 변경한다. 그 후에 1차원 배열 형태인 각 band 데이터를 75 x 75 크기의 2차원 행렬(이미지)로 변환하는 것이다.

3. 이렇게 각 샘플에 대한 2차원 75 x 75 이미지의 정보가 담긴 X_band_1, X_band_2 변수 2개가 생성되었다. 그런 후, 이 두 개의 데이터를 합쳐 X_train 데이터를 만드는 과정이

```
X_train = np.cocatenate([X_band_1][:,:,:, np.newaxis], X_band_2[:,:,:, np.newaxis], ((X_band_1 + X_band_2) / 2)[:, :, :, np.newaxis]], axis = -1)
```
이다.

```X_band_1[:,:,:, np.newaxis]```
를 먼저 살펴보자. X_band_1은 (N, 75, 75)형태이다. 각각의 N개의 샘플에 대해, 75 x 75 배열의 정보를 갖고 있다. np.newaxis는 배열에 새로운 축을 추가한다. 여기서는 마지막에 축을 추가하여, (N, 75, 75, 1) 형태로 만든다. 이는 각 75 x 75 이미지를 '채널 1개짜리 이미지'로 취급하기 위함이다. (컬러 이미지의 RGB 채널처럼 말이다.)

X_band_2도 마찬가지로 작업을 해준다.

그런 후, (X_band_1 + X_band_2) / 2를 통해 각 픽셀별 평균값을 계산해 준다. 이 또한 (N, 75, 75) 형태의 배열이 되는데, 이것이 세번 째 밴드(채널)이 된다. 이 채널 또한 (N, 75, 75, 1)의 형태로 변환해준다. 

이러한 3개의 밴트를 리스트로 담은 후, np.concatenate([...])에 파라미터로 넣은 후, axis = -1(마지막 축)을 기준으로 합치라는 의미이다.

결과적으로 X_train은 (N, 75, 75 ,3) 형태의 4차원 배열이 된다. 이는, N개의 샘플, 각 샘플의 크기는 75x75 크기이며, 3개의 채널, HH, HV, (HH+HV) / 2 를 갖는 이미지 데이터임을 의미한다. 딥러닝에서 이미지 데이터를 다룰 때 흔히 볼 수 있는 형태이다.

요약 : HH, HV, HH&HV 평균을 각각 하나의 채널로 하여, 3개의 채널을 가진 75 x 75 크기의 이미지 데이터셋을 만듦. 이는 마치 컬러 이미지 (RGB 채널)처럼 모델이 각기 다른 특징을 학습할 수 있도록 데이터를 구성하는 것이다.

Q. np.newaxis가 배열에 새로운 축을 추가한다는 것이 무슨 의미인가?
- 원래의 데이터는 (N, 75, 75), 이미지의 총 N개수(샘플 수), 75 높이, 75 너비였다. 이것은 마치 N장의 흑백 사진(75 x 75)이 겹쳐있는 앨범과 같다. 각 사진은 그 자체로 하나의 완전한 이미지이다. np.newaxis를 사용하면, (N, 75, 75, 1)이 되는데, 1은, 각 이미지가 가진 채널의 수를 의미한다. 즉, N개의 각 75x75 이미지에, 이 이미지는 1개의 채널로 구성되어 있습니다, 라는 꼬리표를 붙여주는 것과 같다.

컬러 이미지와의 비교

- 컬러 이미지 : 보통 RGB의 세 가지 색상 정보를 합쳐서 하나의 색을 표현한다. 때문에, 컬러 이미지 하나의 모양은 (높이, 너비, 3)이 된다. 여기서 3이 바로 채널 수, (R,G,B)를 의미한다.

- 흑백 이미지: 흑백 이미지는 색상 정보 없이 밝기 정보만으로 이루어져 있다. 그래서 원래는 (75 x 75)만으로도 표현할 수 있는데, 이미지 처리 라이브러리나 딥러닝 모델은 입력 데이터의 형식을 통일하기 위해, 흑백 이미지에도 채널 개념을 명시적으로 요구한다. 때문에, (높이, 너비, 1) 형태로 만들어주는 것이다. 여기서 1은 "밝기 정보 채널 하나"를 의미한다!

결론 : (N, 75, 75)에서 (N, 75, 75, 1)로 바뀌는 것은, N개의 75 x 75 이미지 각각에 "나는 단일 채널(흑백 밝기 정보)로 이루어진 이미지야"라는 정보를 명시적으로 부여하는 것이다!

Q. 그렇다면, 현재의 X_train은 (N, 75, 75, 3)구조인데, 이러한 것은 컬러 사진이라고 이해해도 될까?
- 구조적으로는 컬러 사진과 유사하다. 현재 데이터에서는 위색 합성 이미지라고 이해하면 된다.
1. 첫 번째 채널 : HH 편파 레이더의 후방 산란 강도
2. 두 번째 채널 : HV 편파 레이더의 후방 산란 강도
3. 세 번째 채널 : HH와 HV값의 평균

In [6]:
# Take a look at a iceberg

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected = True)
def plotmy3d(c, name):

    data = [
        go.Surface(
            z = c
        )
    ]
    layout = go.Layout(
        title = name,
        autosize = False,
        width = 700,
        height = 700,
        margin = dict(
            l = 65,
            r = 50,
            b = 65,
            t = 90
        )
    )
    fig = go.Figure(data = data, layout=layout)
    py.iplot(fig)
plotmy3d(X_band_1[12, :, :], 'iceberg')

```
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected = True)
```
를 먼저 한 번 살펴보자.

1. `import plotly.offline as py`
- plotly는 인터랙티브, 상호작용이 가능한 그래프를 만들 수 있는 강력한 시각화 라이브러리이다.
- plotly.offline은 인터넷 연결 없이, 주로 주피터 노트북과 같은 환경에서도 Ploty 그래프를 직접 표시할 수 있게 해주는 모듈이다.

2. `import plotly.graph_objs as go`
- plotly.graph_objs 모듈은 그래프를 구성하는 다양한 요소들, 예를 들어 선 그래프, 막대 그래프, 표면 그래프, 레이아웃 설정 등을 **객체 형태**로 제공한다.

3. `py.init_notebook_mode(connected = True)`
- 주피터 노트북 환경에서 Plotly 오프라인 모드를 초기화 하는 함수이다. 이 함수를 실행해야, 노트북 내에서 Plotly 그래프가 제대로 그려진다!

4. `def plotmy3d(c, name):`
- plotmy3d라는 이름의 함수를 정의한다. 이 함수는 두 개의 인자를 받는다.
    - c는 시각화할 2차원 배열 데이터이다. 이 값들이 3D 표면의 높이 (z)축 값이 된다.
    - name : 그래프의 제목으로 사용될 문자열

5. `data = [ go.Surface(z = c)]`
- 그래프에 그려질 실제 데이터를 정의하는 부분이다. Plotly에서는 하나 이상의 "trace"를 리스트 형태로 data 변수에 담는다.
- `go. Surface(...)` : 3D 표면 그래프에 나타내는 Surface 객체를 생성한다.
    -  z = c : Surface 객체의 z 매개변수에 함수 입력으로 받은 2차원 배열 c를 할당한다. 이는 c 배열이 각 요소 값이 해당 위치에서의 표면 높이를 결정한다는 의미이다. x와 y 좌표는 c배열의 행과 열 인덱스에 따라, 자동을 ㅗ격차 형태로 생성된다. (예 : 75 x 75 배열이라면, x,y는 각각 0~74 범위)
 
6. layout = go.Layout(...)
- 그래프의 전체적인 모양, 제목, 크기, 여백 등 시각적인 요소들을 설정하는 Layout 객체를 생성한다
    - title = name : 그래프의 제목을 함수 입력으로 받은 name 문자열로 설정
    - autosize = False : 그래프 크기가 자동으로 조절되는 것을 막음
    - width = 700, height = 700 : 그래프의 가로 및 세로 크기를 700 픽셀로 지정.
    - margin = dict(...) : 그래프의 영역 주변의 여백을 설정 (1:left, r : right, b : bottom, t : top)


7. fig = go.Figure(data = data, layout = layout)
- 앞서 정의한 data(실제 그려질 데이터)와 layout(그래프의 모양 설정)을 합쳐서 하나의 figure 객체를 만든다. 이 figure 객체가 바로 우리가 그리려는 최종 그래프이다!

8. py.iplot(fig)
- 생성된 Figure 객체 (fig)를 화면에 그린다. iplot 함수는 plotly.offline 모듈에 있으며, 주로 주피터 노트북과 같은 인터랙비트 환경에 그래프를 표시할 때 사용된다.

9. plotmy3d(X_band_1[12, :, :], 'iceberg')
- 앞서 정의한 plotmy3d 함수를 실제로 호출하여, 그래프를 그리는 부분.
    - X_band_1은 HH 편파 데이터를 담고 있는 (N, 75, 75) 형태의 넘파이 배열
    - `[12, :, :]` 는 13번째 샘플에 대한 75x75 이미지(파편 정보).
    - 결과적으로 X_band_1의 13번째 이미지에 해당하는 75x75 크기의 2차원 배열이 plotmy3d 함수의 c 인자로 전달됩니다. 이 배열의 값들이 3D 표면의 높이가 됩니다.


Q1. go를 통해 객체를 생성한다는 것이 무슨 의미인가? layout, data 객체를 생성하면, 그 안에 title, autosize 등의 attribute값을 설정해주는 것이라고 이해해도 되나?

- 맞음! plotly.graph_objs를 go라는 이름으로 가져왔다. 이 go 모듈 안에는 다양한 종류의 그래프 구성 요소들(Surface, Layout, Scatter, Bar 등)이 class 형태로 미리 정의되어있다!
    - go.Layout(...)이나 go.Surface(...) 코드를 실행하는 것은, 이 클래스라는 설계도를 바탕으로 실제 메모리상에 객체 또는 인스턴스(Layout, data 등)를 만들어내는 과정.
    - 객체 생성할 때, 괄호 안에 넣어주는 값들 예를 들어, 'go.Layout(title = 'My Plot', autosize = False)'에서 My Plot이나 False같은 것들은 생성되는 객체가 가지게 될, 구체적인 속성 또는 특성을 설정하는 것이다!
    - 붕어빵 틀(클래스)는 팥앙금 종류, 크기 같은 속성을 가질 수 있도록 설계되어 있고, 실제 붕어빵(객체)를 만들 때, 팥앙금 종류는 단팥, 크기는 대형!과 같이 구체적인 값을 지정해주는 것과 같다!

Q2. 이 부분이 잘 이해가 안 가. c는 (N, 75 x 75)이고 우리는 z에 c를 배정하도록 해서 data(우리가 그릴 데이터)를 설정했어. 그렇다면, x랑 y좌표만 설정이 되어있는데, 어디서 z좌표가 설정이 되는거야? (plotmy3d(c, name))에 대한 설명. (그래프에 대한 직관적인 이해) 그렇다면, 일단, 75 x 75 짜리 공간 위에다가 해당 높이에는 그 레이저의 강도 값을 놓는다는거지? 그렇다면, 여기에서는 레이저의 반사율? 편파값이 3D에서(또는 어떠한 현실세계에서) 어떤 특정한 높이를 반영한다고 이해해도 될까?

1. 75 x 75 짜리 공간 위ㅔ아다가 해당 높이에 그 레이저의 강도값을 놓는다는 것은 매우 정확! 75 x 75 칸으로 이루어진 바둑판이 있고, 각 칸마다 그 위치에서의 레이더 신호 반사 강도를 나타내는 막대기를 수직으로 꽂는 것!

2. "여기에서는" 레이저의 편파값이 3D에서 또는 어떠한 현실 세계 물리 공간에서 특정한 높이를 반영한다고 이해해도 될까?
- 이 부분은 조금 더 신중한 이해가 필요. 결론부터 말하자면, 그래포에서의 높이가, 반드시 현실 세계의 물리적인 높이를 직접적으로 의미하는 것은 아님.
    - 그래프의 높이 = 레이저 신호의 반사 강도 : 레이더 신호가 해당 지표면에 부딪혀 얼마나 강하게 위성으로 되돌아왔는지를 나타내는 값. 이 반사 강도에 영향을 미치는 요인은 매우 다양함.
    - 표면의 거칠기 : 일반적으로 거친 표면은 신호를 여러 방향으로 산란시켜, 위성으로 돌아오는 신호가 강해질 수 있음. 반대로 매끄러운 표면은 신호를 다른 방향으로 반사시켜 위성으로 돌아오는 신호가 약해짐. 예를 들어 물은 HH가 -27로 매우 약했음. 반면에 주거지 같은 settlement는 2로 비교적 강했음.
    - 물체의 재질 : 금속과 같은 우리가 목표로 하는 선박과 얼음을 구별하기 위해서는 반사되는 레이더의 신호가 매우 다를 것임.
    - 그 외의 수분 함량, 기하학적 구조, 입사각 등에 따라 다를 것으로 예상!

결론 :
- 3D 표면 그래프에서의 높이는, 레이더가  특정지점을 얼마나 밝게 또는 강하게 인식했는지를 시각화.



That's a cool looking iceberg we have. Remember, in radar data, the shape of the iceberg is going to be like a mountain as shown in here. Since t his is not a actual image but scatter from radar, the shape is going to have peaks and distortions like these. The shape of the ship is going to be like a point, maybe like a elongated point. From here the structual differences arise and we can exploit those differences using a CNN. It would be helpful if we can create composite(합성물) images using the backscatter from radar.

In [7]:
plotmy3d(X_band_1[14,:,:], 'Ship')

That's a ship, looks like a elongated point. We don't have much resolution in images to visualize the shape of the ship. However CNN is here to help. There are few papers on ship iceberg classification like this:

http://elib.dlr.de/99079/2/2016_BENTES_Frost_Velotto_Tings_EUSAR_FP.pdf

However their data have much better resolution so I don't feel that the CNN they used would be suitable here.


Get back to building a CNN using Keras. Much better frameworks then others. You will enjoy for sure

In [8]:
#Import Keras
from matplotlib import pyplot

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import BatchNormalization # 'normalization' 하위 모듈이 아닌 'layers'에서 직접 가져옴
from tensorflow.keras.layers import Concatenate       # 'merge' 하위 모듈이 아닌 'layers'에서 직접 가져옴
from tensorflow.keras.models import Model
from tensorflow.keras import initializers
from tensorflow.keras.optimizers import Adam # 또는 최신 Keras 3.x 에서는 keras.optimizers.Adam
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, LearningRateScheduler # 학습률 스케줄러 콜백
from tensorflow.keras.optimizers.schedules import ExponentialDecay

2025-06-12 06:22:26.442996: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749709346.863608      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749709346.977837      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


각 라이브러리들에 대한 설명


In [9]:
#define our model

def getModel():
    #Building the Model
    gmodel = Sequential()
    #Conv Layer 1
    gmodel.add(Conv2D(64, kernel_size=(3,3), activation = 'relu', input_shape=(75, 75, 3)))
    gmodel.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 2
    gmodel.add(Conv2D(128, kernel_size=(3,3), activation = 'relu'))
    gmodel.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 3
    gmodel.add(Conv2D(128, kernel_size=(3,3), activation = 'relu'))
    gmodel.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 4
    gmodel.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))
    gmodel.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    gmodel.add(Dropout(0.2))

    #Flatten the data for upcoming dense layers
    gmodel.add(Flatten())

    #Dense Layers
    gmodel.add(Dense(512))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Dense Layer 2
    gmodel.add(Dense(256))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Sigmoid Layer
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))

    myoptim = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    gmodel.compile(loss='binary_crossentropy',
                   optimizer=myoptim,
                   metrics=['accuracy'])
    gmodel.summary()
    return gmodel

def get_callbacks(filepath, patience=2):
    es = EarlyStopping(monitor='val_loss', patience=patience, mode="min") # monitor 명시 권장
    # ModelCheckpoint 수정: save_weights_only=True 추가
    msave = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, save_weights_only=True) # monitor 명시 권장
    return [es, msave]

file_path = ".model_weights.weights.h5" # 이 파일 이름은 가중치만 저장할 때 적합
callbacks = get_callbacks(filepath=file_path, patience=5)

코드 설명 

1. getModel() 함수
- 이 getModel() 함수는 이미지 분류를 위한 CNN 모델을 정의하고 반환

- gmodel = Sequential()
    - Keras의 Sequential 모델 객체를 생성하여 gmodel 변수에 할당.
    - 딥러닝 개념 (Sequential 모델) : Sequential 모델은 레이어들을 차곡차곡 순차적으로 쌓아 올리는 가장 간단한 형태의 딥러닝 모델!
- Conv Layer (첫 번째 컨볼루션 블록)
    - `gmodel.add(Conv2D(64, kernel_size = (3,3), activation = 'relu', input_shape = (75, 75, 3)))`
    - gmodel에 2D CV Layer를 추가!
    - 64 : 이 레이어에서 사용할 컨볼루션 필터의 개수. 각  필터는 이미지에서 서로 다른 특징을(선, 모서리, 질감)을 감지하도록 학습!
    - kernel_size = (3,3) : CV 필터의 크기. 3x3픽셀 영역을 한 번에 보면서, 연산을 수행!
    - activation = 'relu'
    - input_shape = (75,75, 3) : 모델의 첫 번째 레이어에만 지정하는 입력 데이터의 형태.
    - 딥러닝 개념 (Conv2D, 컨볼루션 연산) : 입력 이미지에 필터를 적용하여 슬라이딩하면서 합성곱 연산을 수행. 이를 통해, 이미지의 공간적인 특징을 추출한 "특징 맵(feature map)"을 생성
-  `gmodel.add(MaxPoolig2D(pool_size = (3,3), strides=(2,2)))`
    - 2D 맥스 풀링 레이어를 추가
    - `pool_size = (3,3)` : 풀링 윈도우의 크기. 3x3 영역 내에서 가장 큰 max값 하나만을 선택!
    - strides = (2,2) 풀링 윈도우를 2픽셀식 이동!
    - 딥러닝 개념(Max pooling, 풀링) : 특징 맵 feature map의 크기를 줄여(다운샘플링) 계산량을 감소시키고, 주요 특징만 강조하며, 객체의 위치가 약간 변하더라도 동일하게 인식할 수 있는 능력(translation invariance)을 높이는 데 도움을 줌.
- gmodel.add(Dropout(0.2))
    - 각 샘플에 의해 훈련될 때마다, 이전 레이어의 출력 중 20%의 뉴런을 무작위로 비활성화!
    - 딥러닝 개념(Dropout, 규제) : 모델이 훈련 데이터에 너무 과도하게 맞춰지는 현상(overfitting)을 방지하기 위해 규제 기법 중 하나. 뉴런 의존도를 줄여, 하이 베리언스를 방지

- 유사한 방식으로 Conv Layer (CNN -> Pooling)를 총 4개 만듦.

- `gmodel.add(Flatten())`
    - 플래튼 레이어 flatten 레이러를 추가.
    - 딥러닝 개념 (Flatten) : 이전 컨볼루션/풀링 레이어를 통과한 다차원의 특징을 1차원의 긴 벡터로 쭉 펼쳐, 이후에 나올 완전 연결 계층 (Dense Layer)의 인풋값으로 사용할 준비!

- Dense Layer 1
    - `gmodel1.add(Dense(512, activation = 'relu'))`
        -  512개의 뉴런을 가진 완전 연결 레이어를 추가. 활성화 함수로는 ReLu.
        -  딥러닝 개념 (Dense Layer, 완전 연결 계층) : 이전 레이어의 모든 출력과 현재 레이어의 모든 뉴런이 서로 연결된 형태. 컨볼루션 계층에서 추출된 특징들을 종합하여, 고차원적인 패턴을 학습하고 분류를 위한 준비를 함.
- Dense Layer 2
    - 동일하게 생성
 
- Output Layer(출력 계층)
    - `gmodel.add(Dense(1, activation = 'sigmoid'))`
        - 1개의 뉴런을 가진 완전 연결 레이어를 출력층으로 추가. 활성화 함수로는 시그모이드를 사용.
        - 딥러닝 개념 : 바이너리 분류이기 때문에, 시그모아드로 해석 가능.
 
- Optimizer (옵티마이저 설정)
    - `myoptim = LegacyAdam(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, decay = 0.0) `
        - 딥러닝 개념 (optimizer, Adam) : 모델의 예측값과 실제값 사이의 오차(손실)을 최소화하도록 모델 내부의 학습 가능한 파라미터(가중치와 편향)을 업데이트 하는 알고리즘입니다. (백프로페게이션 자동화).
        - learning_rate : 학습률. 한 번의 업데이트에서 파라미터를 얼마나 변경할지 결정하는 값.
        - beta_1, beta_2, epsilon : Adam 알고리즘의 내부 하이퍼 파라미터
        - decay : 매 업데이트마다 학습률을 점진적으로 줄이는 값.
 
- Compile (모델 컴파일)
    - `gmodel.comile(loss = 'binary_crossentropy, optimizer = myoptim, metrics = ['accuracy'])`
        - 모델의 학습 과정을 설정(컴파일)합니다.
        - loss = 'binary_crossentropy' : 손실함수를 지정. 이진 분류 문제에 적합한 교차 엔트로피 손실 함수.
        - optimizer = myoptim : 위에서 설정한 Adam 옵티마이저 사용
        - metrics = ['accuracy'] : 훈련 및 평가 과정에서 모델의 성능을 측정할 지표로 정확도 사용
 
- gmodel.summary()
    - 이미 생성된 모델의 전체 구조를 요약하여 출력. 각 레이어의 이름, 출력 형태 (output shape), 파라미터 수 등을 보여주어, 모델이 의도대로 구성되었는지 확인하는데 유용.
 
- return gmodel
    - 컴파일까지 완료된 모델 객체 gmodel을 반환!
 
2. get_callbacks() 함수
- `def get_callbacks(filepath, patience = 2)`
    - get_callbacks 라는 이름의 함수를 정의. filepath(모델 저장 경러로) 

In [10]:
target_train = train['is_iceberg']
X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train, random_state = 1, train_size = 0.75)

In [11]:
# without denoising, core features

import os
gmodel=getModel()
gmodel.fit(X_train_cv, y_train_cv,
          batch_size=24,
          epochs=50,
          verbose=1,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

I0000 00:00:1749709371.094676      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1749709371.095340      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 73, 73, 64)          │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 15, 15, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 5, 5, 64)            │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 2, 2, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 2, 2, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             257 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 1)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 560,193 (2.14 MB)

 Trainable params: 560,193 (2.14 MB)

...

Epoch 1/50


I0000 00:00:1749709378.175951      67 service.cc:148] XLA service 0x7c7b04019b10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1749709378.177389      67 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1749709378.177430      67 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1749709378.726739      67 cuda_dnn.cc:529] Loaded cuDNN version 90300


13/51 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5261 - loss: 3.1862

I0000 00:00:1749709385.377889      67 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


51/51 ━━━━━━━━━━━━━━━━━━━━ 19s 141ms/step - accuracy: 0.5190 - loss: 1.7222 - val_accuracy: 0.6683 - val_loss: 0.5708
Epoch 2/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6672 - loss: 0.5698 - val_accuracy: 0.6185 - val_loss: 0.5498
Epoch 3/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6906 - loss: 0.5266 - val_accuracy: 0.7132 - val_loss: 0.5168
Epoch 4/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7465 - loss: 0.5099 - val_accuracy: 0.8080 - val_loss: 0.4745
Epoch 5/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7922 - loss: 0.4451 - val_accuracy: 0.8005 - val_loss: 0.4558
Epoch 6/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8061 - loss: 0.4372 - val_accuracy: 0.8055 - val_loss: 0.4388
Epoch 7/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7988 - loss: 0.4304 - val_accuracy: 0.8055 - val_loss: 0.4125
Epoch 8/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8178 - loss: 0.3932 - val_accuracy: 0.8254 - val_loss: 

In [12]:
gmodel.load_weights(filepath = file_path)
score = gmodel.evaluate(X_valid, y_valid, verbose = 1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8958 - loss: 0.2888  
Test loss: 0.28347209095954895
Test accuracy: 0.895261824131012


In [13]:

X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , ((X_band_test_1+X_band_test_2)/2)[:, :, :, np.newaxis]], axis=-1)
predicted_test=gmodel.predict(X_test)

264/264 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step


In [14]:
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('sub.csv', index=False)